# Data Preprocessing

* Data cleaning: handle missing values, duplicates, inconsistant or invalid vallues, outliers

* Data reduction: reduce number of attributes, reduce number of attribute values

* Data transformation: attribute construction, normalization

* Data discretization: encode to numerical attributes

## Setting up the Notebook

### Import Required Packages

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

---

## Loading the Data

In [2]:
# Load file into pandas dataframe
df = pd.read_csv('../data/train.csv')
# df = pd.read_csv('../data/test.csv')

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points, each with 30 attributes.


---

## Data Cleaning

Before data cleaning, remove the known attributes that are not meaningful to our prediction model:
  * Meaningless idendifier: listing_id 
  * Attributes in free text: title, description, features, accessories
  * Attribute with the same value: eco_category, indicative_price
  * Attribute unlikely to affect price: curb_weight

In [3]:
columns_to_drop = [
    'listing_id',          # Meaningless identifier
    'title',               # Attributes in free text
    'description',
    'features',
    'accessories',
    'eco_category',        # Attribute with the same value
    'indicative_price',
    'curb_weight',         # Attribute unlikely to affect price

    'original_reg_date',
    'lifespan',
]

df = df.drop(columns=columns_to_drop)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points, each with 20 attributes.


### Handle Missing Values
Firstly, for each of the columns with missing value, check the number of rows with NaN values.
There are 3 scenarios:
1. NaN values are valid, no need to remove.
2. NaN values are not valid, data points are big (e.g. fuel_type has 19121 rows with NaN values), set a default value.
3. NaN values are not valid, data points are small. Remove the data points with attribute with NaN values that are known to be invalid. 

In [4]:
columns_to_check = [
    'make',
    'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf'
]

# Calculate the number of NaN values in each specified column
nan_counts = df[columns_to_check].isna().sum()

# Print the number of NaN values for each column
for column, count in nan_counts.items():
    print(f"Column '{column}' has {count} rows with NaN values.")

Column 'make' has 1316 rows with NaN values.
Column 'fuel_type' has 19121 rows with NaN values.
Column 'manufactured' has 7 rows with NaN values.
Column 'power' has 2640 rows with NaN values.
Column 'engine_cap' has 596 rows with NaN values.
Column 'mileage' has 5304 rows with NaN values.
Column 'no_of_owners' has 18 rows with NaN values.
Column 'depreciation' has 507 rows with NaN values.
Column 'road_tax' has 2632 rows with NaN values.
Column 'dereg_value' has 220 rows with NaN values.
Column 'omv' has 64 rows with NaN values.
Column 'arf' has 174 rows with NaN values.


In [5]:
columns_with_invalid_NaN= [
    'make',
    # 'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf'
]

df = df.dropna(subset=columns_with_invalid_NaN)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 17025 data points, each with 20 attributes.


In [6]:
# Replace NaN values of attribute fuel_type with 'petrol'
df['fuel_type'] = df['fuel_type'].fillna('petrol')

### Remove Exact Duplicates

In [7]:
df = df.drop_duplicates()

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 17021 data points, each with 20 attributes.


### Transform categorical value to numerical values

In [8]:
categorical_columns = [
    'make',
    'model',
    'type_of_vehicle',
    'transmission',
    'fuel_type',
    'opc_scheme',
]

le = LabelEncoder()
for column in categorical_columns:
  df[column] = le.fit_transform(df[column])

### Transform date time attributes to numerical values

In [9]:
df['reg_date'] = pd.to_datetime(df['reg_date'], format='%d-%b-%Y')
df['reg_year'] = df['reg_date'].dt.year
df = df.drop(columns=['reg_date'])

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 17021 data points, each with 20 attributes.


### Handle category attribute

In [10]:
# Replace '-' with an empty string
df['category'] = df['category'].replace('-', '')

# Split the 'category' column into lists
df['category_list'] = df['category'].str.split(', ')

# Handle empty strings by replacing them with empty lists
df['category_list'] = df['category_list'].apply(lambda x: [] if x == [''] else x)

# Import itertools for flattening lists
from itertools import chain

# Flatten the list of lists to a single list
all_categories = list(chain.from_iterable(df['category_list']))

# Get the unique categories
unique_categories = set(all_categories)

# Print the number of unique categories
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the category lists
category_dummies = mlb.fit_transform(df['category_list'])

# Create a DataFrame with the one-hot encoded categories
category_df = pd.DataFrame(category_dummies, columns=mlb.classes_, index=df.index)

# Concatenate the new dummy columns to the original DataFrame
df = pd.concat([df, category_df], axis=1)

# Drop the temporary 'category_list' column if desired
df.drop('category_list', axis=1, inplace=True)
df.drop('category', axis=1, inplace=True)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

Number of unique categories: 14
Unique categories: {'rare & exotic', 'opc car', 'vintage cars', 'consignment car', 'sta evaluated car', 'parf car', 'direct owner sale', 'coe car', 'sgcarmart warranty cars', 'hybrid cars', 'premium ad car', 'low mileage car', 'imported used vehicle', 'almost new car'}
There are 17021 data points, each with 33 attributes.


---

## Saving the Data

In [11]:
file_name = '../data/preprocessed/train_preprocessed.csv'
# file_name = '../data/preprocessed/test_preprocessed.csv'


# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file '../data/preprocessed/train_preprocessed.csv' has been deleted.
DataFrame has been saved to '../data/preprocessed/train_preprocessed.csv'.
